In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
import os
from glob import glob
from PIL import Image

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir='/content/drive/My Drive/archive'
df = pd.read_csv(os.path.join(data_dir, 'hmnist_28_28_RGB.csv'))

In [ ]:
ldf = pd.read_csv(os.path.join(data_dir, 'HAM10000_metadata.csv'))

In [ ]:
ldf.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
ldf['image_id'].shape

(10015,)

In [ ]:
df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [ ]:
image_dir_part_1 = '/content/drive/My Drive/archive/HAM10000_images_part_1'
image_dir_part_2 = '/content/drive/My Drive/archive/HAM10000_images_part_2'


In [ ]:
import os
import numpy as np
from PIL import Image

def load_images_from_directory(directory, desired_height, desired_width):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            img = Image.open(os.path.join(directory, filename))
            img = img.resize((desired_width, desired_height))  # Resize to desired dimensions
            img = np.array(img)
            images.append(img)
    return np.array(images)

# Set your desired image dimensions
desired_width = 256
desired_height = 192
channels = 3  # Assuming RGB images

# Load and resize images from both directories
images_part_1 = load_images_from_directory(image_dir_part_1, desired_height, desired_width)
images_part_2 = load_images_from_directory(image_dir_part_2, desired_height, desired_width)

# Stack the images into a single array with shape (no. of samples, height, width, channels)
all_images = np.vstack([images_part_1, images_part_2])

# Save the NumPy array as a .npy file if needed
np.save('all_images.npy', all_images)



In [ ]:
all_images = np.load('all_images.npy')

In [ ]:
all_images.shape

(10056, 192, 256, 3)

In [ ]:
all_images.shape, images_part_1.shape

((10056, 192, 256, 3), (5028, 192, 256, 3))

In [ ]:
lesion_danger = {
    'nv': np.array([1,0,0,0,0,0,0]),
    'mel': np.array([0,1,0,0,0,0,0]),
    'bkl': np.array([0,0,1,0,0,0,0]),
    'bcc': np.array([0,0,0,1,0,0,0]),
    'akiec': np.array([0,0,0,0,1,0,0]),
    'vasc': np.array([0,0,0,0,0,1,0]),
    'df': np.array([0,0,0,0,0,0,1])
}

ldf["dxn"] = ldf["dx"].map(lesion_danger.get)

In [ ]:
ldf['dxn'].shape

(10015,)

In [ ]:
import os
import numpy as np
from PIL import Image

image_dir_part_1 = '/content/drive/My Drive/archive/HAM10000_images_part_1'
image_dir_part_2 = '/content/drive/My Drive/archive/HAM10000_images_part_2'

def load_labels_from_directory(directory):
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
          labels.append(ldf.loc[ldf['image_id'] == filename[:12], 'dxn'].iloc[0])
    return labels

labels_1 = load_labels_from_directory(image_dir_part_1)
labels_2 = load_labels_from_directory(image_dir_part_2)

# Stack the images into a single array with shape (no. of samples, height, width, channels)
all_labels = np.concatenate([labels_1, labels_2])

# Save the NumPy array as a .npy file if needed
np.save('all_labels.npy', all_labels)

In [ ]:
all_labels = np.load('all_labels.npy')

In [ ]:
all_labels.shape

(10056, 7)

In [ ]:
labels = all_labels


In [ ]:
labels.shape

(10056, 7)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np

# Specify the full file path to all_samples.npy
file_path = '/content/all_images.npy'

# Load the data from the specified file path
data = np.load(file_path)


# Load the data from all_samples.npy
labels_path='/content/all_labels.npy'

# Load labels or classes from a corresponding file (e.g., labels.npy)
labels = np.load(labels_path)

ndata = data[:1000]
nlabels = labels[:1000]
# Split the data and labels into training and testing sets
test_size = 0.2  # You can adjust the test size as needed
x_train, x_test, y_train, y_test = train_test_split(ndata, nlabels,test_size=test_size, random_state=42)

# Now x_train, x_test, y_train, and y_test contain the training and testing data and labels.


In [ ]:

x_train.shape, x_test.shape,y_train.shape, y_test.shape

((800, 192, 256, 3), (200, 192, 256, 3), (800, 7), (200, 7))

In [ ]:
np.save('x_train.npy',x_train)
np.save('x_test.npy',x_test)
np.save('y_train.npy',y_train)
np.save('y_test.npy',y_test)

In [ ]:
x_test=np.load("/content/x_test.npy")
y_test=np.load("/content/y_test.npy")
x_train=np.load("/content/x_train.npy")
y_train=np.load("/content/y_train.npy"

# VGG 19

In [ ]:
from keras.applications import VGG19

In [ ]:
import os
from glob import glob
from PIL import Image

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
pre_trained_model = VGG19(input_shape=(192, 256, 3), include_top=False, weights="imagenet")

80134624/80134624 [==============================] - 3s 0us/step


In [ ]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False

print(len(pre_trained_model.layers))

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
22


In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 6, 8, 512)


In [ ]:
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
from tensorflow.keras.optimizers.legacy import Adam

optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

NameError: name 'layers' is not defined

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 192, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 192, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 192, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 96, 128, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 96, 128, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 96, 128, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 48, 64, 128)       0     

In [ ]:
import keras

nmodel = keras.Sequential([
  keras.layers.Conv2D(input_shape=(192,256,3), filters=4, kernel_size=1,
                      strides=1, activation='relu', name='Conv1'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
  keras.layers.Conv2D( filters=8, kernel_size=1,
                      strides=1, activation='relu', name='Conv2'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
  keras.layers.Conv2D( filters=16, kernel_size=1,
                      strides=1, activation='relu', name='Conv3'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
  keras.layers.Conv2D( filters=64, kernel_size=1,
                      strides=1, activation='relu', name='Conv4'),
  keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
  keras.layers.Conv2D( filters=64, kernel_size=1,
                      strides=1, activation='relu', name='Conv5'),
   keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
  keras.layers.Conv2D( filters=64, kernel_size=1,
                      strides=1, activation='relu', name='Conv6')
])

nmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 192, 256, 4)       16        
                                                                 
 max_pooling2d (MaxPooling2  (None, 96, 128, 4)        0         
 D)                                                              
                                                                 
 Conv2 (Conv2D)              (None, 96, 128, 8)        40        
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 64, 8)         0         
 g2D)                                                            
                                                                 
 Conv3 (Conv2D)              (None, 48, 64, 16)        144       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 24, 32, 16)        0

In [ ]:
nlast_layer = nmodel.get_layer('Conv6')
print('last layer output shape:', nlast_layer.output_shape)
nlast_output = nlast_layer.output

last layer output shape: (None, 6, 8, 64)


In [ ]:
nx = layers.GlobalMaxPooling2D()(nlast_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
nx = layers.Dense(512, activation='relu')(nx)
# Add a dropout rate of 0.5
nx = layers.Dropout(0.5)(nx)
# Add a final sigmoid layer for classification
nx = layers.Dense(7, activation='softmax')(nx)

# Configure and compile the model

nnmodel = Model(nmodel.input, nx)
from tensorflow.keras.optimizers.legacy import Adam

optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
nnmodel.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [ ]:
nnmodel.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1_input (InputLayer)    [(None, 192, 256, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 192, 256, 4)       16        
                                                                 
 max_pooling2d (MaxPooling2  (None, 96, 128, 4)        0         
 D)                                                              
                                                                 
 Conv2 (Conv2D)              (None, 96, 128, 8)        40        
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 64, 8)         0         
 g2D)                                                            
                                                                 
 Conv3 (Conv2D)              (None, 48, 64, 16)        144 

In [ ]:
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        shear_range = 10)

#crashing the code

In [ ]:
train_datagen.fit(x_train)

In [ ]:
val_datagen = ImageDataGenerator()


In [ ]:
val_datagen.fit(x_test)

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = True

In [ ]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5,
                                            min_lr=0.000001, cooldown=2)

In [ ]:
nnmodel.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [ ]:

nnmodel.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1_input (InputLayer)    [(None, 192, 256, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 192, 256, 4)       16        
                                                                 
 max_pooling2d (MaxPooling2  (None, 96, 128, 4)        0         
 D)                                                              
                                                                 
 Conv2 (Conv2D)              (None, 96, 128, 8)        40        
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 64, 8)         0         
 g2D)                                                            
                                                                 
 Conv3 (Conv2D)              (None, 48, 64, 16)        144 

In [ ]:
#dont run
batch_size = 64
epochs = 35
history = model.fit_generator(train_datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(x_test, y_test),
                              verbose = 1, steps_per_epoch=(x_train.shape[0] // batch_size),
                              validation_steps=(x_test.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

<ipython-input-44-91d53d4fe729>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(x_train,y_train, batch_size=batch_size),


Epoch 1/35
12/12 [==============================] - 2015s 167s/step - loss: 4.3630 - acc: 0.5367 - val_loss: 1.1025 - val_acc: 0.7292 - lr: 1.0000e-04
Epoch 2/35
12/12 [==============================] - 1957s 163s/step - loss: 1.2039 - acc: 0.6603 - val_loss: 0.9981 - val_acc: 0.6875 - lr: 1.0000e-04
Epoch 3/35
12/12 [==============================] - 1919s 160s/step - loss: 1.1391 - acc: 0.6590 - val_loss: 1.0889 - val_acc: 0.6354 - lr: 1.0000e-04
Epoch 4/35
12/12 [==============================] - ETA: 0s - loss: 1.0948 - acc: 0.6562  
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
12/12 [==============================] - 1925s 168s/step - loss: 1.0948 - acc: 0.6562 - val_loss: 1.0514 - val_acc: 0.7083 - lr: 1.0000e-04
Epoch 5/35
 5/12 [===========>..................] - ETA: 18:41 - loss: 1.0304 - acc: 0.6562

WARNING:tensorflow:From <ipython-input-20-b0e0df1b87c2>:7: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.

Epoch 1/35
112/112 [==============================] - 220s 2s/step - loss: 1.4967 - acc: 0.6339 - val_loss: 0.9478 - val_acc: 0.6719 - lr: 1.0000e-04

Epoch 2/35
112/112 [==============================] - 208s 2s/step - loss: 1.0225 - acc: 0.6537 - val_loss: 0.8645 - val_acc: 0.7161 - lr: 1.0000e-04

Epoch 3/35
112/112 [==============================] - 209s 2s/step - loss: 0.9761 - acc: 0.6689 - val_loss: 0.8454 - val_acc: 0.6823 - lr: 1.0000e-04

Epoch 4/35
112/112 [==============================] - 209s 2s/step - loss: 0.9080 - acc: 0.6766 - val_loss: 0.8298 - val_acc: 0.6979 - lr: 1.0000e-04

Epoch 5/35
112/112 [==============================] - 209s 2s/step - loss: 0.8615 - acc: 0.6861 - val_loss: 0.7169 - val_acc: 0.7214 - lr: 1.0000e-04

Epoch 6/35
112/112 [==============================] - 209s 2s/step - loss: 0.8397 - acc: 0.6903 - val_loss: 0.8384 - val_acc: 0.7057 - lr: 1.0000e-04

Epoch 7/35
112/112 [==============================] - 209s 2s/step - loss: 0.8214 - acc: 0.7031 - val_loss: 0.7440 - val_acc: 0.7344 - lr: 1.0000e-04

Epoch 8/35
112/112 [==============================] - 208s 2s/step - loss: 0.7968 - acc: 0.7081 - val_loss: 0.7227 - val_acc: 0.7370 - lr: 1.0000e-04

Epoch 9/35
112/112 [==============================] - 209s 2s/step - loss: 0.7803 - acc: 0.7155 - val_loss: 0.7362 - val_acc: 0.7396 - lr: 1.0000e-04

Epoch 10/35
112/112 [==============================] - 209s 2s/step - loss: 0.7747 - acc: 0.7143 - val_loss: 0.6924 - val_acc: 0.7370 - lr: 1.0000e-04

Epoch 11/35
112/112 [==============================] - 208s 2s/step - loss: 0.7637 - acc: 0.7214 - val_loss: 0.7865 - val_acc: 0.7109 - lr: 1.0000e-04

Epoch 12/35
112/112 [==============================] - ETA: 0s - loss: 0.7714 - acc: 0.7189
Epoch 00012: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
112/112 [==============================] - 208s 2s/step - loss: 0.7714 - acc: 0.7189 - val_loss: 0.7769 - val_acc: 0.7161 - lr: 1.0000e-04

Epoch 13/35
112/112 [==============================] - 209s 2s/step - loss: 0.7058 - acc: 0.7396 - val_loss: 0.6143 - val_acc: 0.7630 - lr: 5.0000e-05

Epoch 14/35
112/112 [==============================] - 209s 2s/step - loss: 0.6832 - acc: 0.7575 - val_loss: 0.6722 - val_acc: 0.7188 - lr: 5.0000e-05

Epoch 15/35
112/112 [==============================] - 209s 2s/step - loss: 0.6702 - acc: 0.7522 - val_loss: 0.6586 - val_acc: 0.7656 - lr: 5.0000e-05

Epoch 16/35
112/112 [==============================] - 209s 2s/step - loss: 0.6584 - acc: 0.7615 - val_loss: 0.6686 - val_acc: 0.7656 - lr: 5.0000e-05

Epoch 17/35
112/112 [==============================] - 209s 2s/step - loss: 0.6391 - acc: 0.7648 - val_loss: 0.6685 - val_acc: 0.7526 - lr: 5.0000e-05

Epoch 18/35
112/112 [==============================] - 209s 2s/step - loss: 0.6164 - acc: 0.7765 - val_loss: 0.5468 - val_acc: 0.7865 - lr: 5.0000e-05

Epoch 19/35
112/112 [==============================] - 209s 2s/step - loss: 0.6035 - acc: 0.7845 - val_loss: 0.6147 - val_acc: 0.7839 - lr: 5.0000e-05

Epoch 20/35
112/112 [==============================] - 209s 2s/step - loss: 0.6111 - acc: 0.7781 - val_loss: 0.5636 - val_acc: 0.7995 - lr: 5.0000e-05

Epoch 21/35
112/112 [==============================] - 209s 2s/step - loss: 0.5908 - acc: 0.7918 - val_loss: 0.5263 - val_acc: 0.8307 - lr: 5.0000e-05

Epoch 22/35
112/112 [==============================] - 209s 2s/step - loss: 0.5585 - acc: 0.8016 - val_loss: 0.6232 - val_acc: 0.7786 - lr: 5.0000e-05

Epoch 23/35
112/112 [==============================] - 209s 2s/step - loss: 0.5568 - acc: 0.8028 - val_loss: 0.5584 - val_acc: 0.8047 - lr: 5.0000e-05

Epoch 24/35
112/112 [==============================] - ETA: 0s - loss: 0.5421 - acc: 0.8048
Epoch 00024: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
112/112 [==============================] - 209s 2s/step - loss: 0.5421 - acc: 0.8048 - val_loss: 0.5664 - val_acc: 0.8047 - lr: 5.0000e-05

Epoch 25/35
112/112 [==============================] - 209s 2s/step - loss: 0.4864 - acc: 0.8265 - val_loss: 0.5589 - val_acc: 0.8099 - lr: 2.5000e-05

Epoch 26/35
112/112 [==============================] - 209s 2s/step - loss: 0.4732 - acc: 0.8317 - val_loss: 0.5743 - val_acc: 0.8151 - lr: 2.5000e-05

Epoch 27/35
112/112 [==============================] - 210s 2s/step - loss: 0.4625 - acc: 0.8317 - val_loss: 0.4844 - val_acc: 0.8229 - lr: 2.5000e-05

Epoch 28/35
112/112 [==============================] - ETA: 0s - loss: 0.4632 - acc: 0.8361
Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
112/112 [==============================] - 209s 2s/step - loss: 0.4632 - acc: 0.8361 - val_loss: 0.5037 - val_acc: 0.8255 - lr: 2.5000e-05

Epoch 29/35
112/112 [==============================] - 209s 2s/step - loss: 0.4219 - acc: 0.8466 - val_loss: 0.5072 - val_acc: 0.8490 - lr: 1.2500e-05

Epoch 30/35
112/112 [==============================] - 209s 2s/step - loss: 0.4114 - acc: 0.8528 - val_loss: 0.4959 - val_acc: 0.8464 - lr: 1.2500e-05

Epoch 31/35
112/112 [==============================] - 209s 2s/step - loss: 0.4009 - acc: 0.8567 - val_loss: 0.5277 - val_acc: 0.8151 - lr: 1.2500e-05

Epoch 32/35
112/112 [==============================] - ETA: 0s - loss: 0.3991 - acc: 0.8570
Epoch 00032: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
112/112 [==============================] - 209s 2s/step - loss: 0.3991 - acc: 0.8570 - val_loss: 0.5298 - val_acc: 0.8307 - lr: 1.2500e-05

Epoch 33/35
112/112 [==============================] - 209s 2s/step - loss: 0.3751 - acc: 0.8637 - val_loss: 0.4735 - val_acc: 0.8333 - lr: 6.2500e-06

Epoch 34/35
112/112 [==============================] - 209s 2s/step - loss: 0.3684 - acc: 0.8676 - val_loss: 0.4809 - val_acc: 0.8516 - lr: 6.2500e-06

Epoch 35/35
112/112 [==============================] - 209s 2s/step - loss: 0.3617 - acc: 0.8693 - val_loss: 0.4922 - val_acc: 0.8438 - lr: 6.2500e-06

In [ ]:
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

Validation: accuracy = 0.835411  ;  loss_v = 0.498586

In [ ]:

batch_size = 64
epochs = 35
history = nnmodel.fit_generator(train_datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(x_test, y_test),
                              verbose = 1, steps_per_epoch=(x_train.shape[0] // batch_size),
                              validation_steps=(x_test.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

<ipython-input-38-639d58d6c316>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = nnmodel.fit_generator(train_datagen.flow(x_train,y_train, batch_size=batch_size),


Epoch 1/35
12/12 [==============================] - 22s 2s/step - loss: 1.9113 - acc: 0.3152 - val_loss: 1.2097 - val_acc: 0.6667 - lr: 1.0000e-04
Epoch 2/35
12/12 [==============================] - 21s 2s/step - loss: 1.3537 - acc: 0.6101 - val_loss: 1.1988 - val_acc: 0.6354 - lr: 1.0000e-04
Epoch 3/35
12/12 [==============================] - 21s 2s/step - loss: 1.2731 - acc: 0.6576 - val_loss: 1.0043 - val_acc: 0.7188 - lr: 1.0000e-04
Epoch 4/35
12/12 [==============================] - 19s 1s/step - loss: 1.2285 - acc: 0.6549 - val_loss: 1.0841 - val_acc: 0.6979 - lr: 1.0000e-04
Epoch 5/35
12/12 [==============================] - 18s 2s/step - loss: 1.2330 - acc: 0.6481 - val_loss: 1.0413 - val_acc: 0.6979 - lr: 1.0000e-04
Epoch 6/35
12/12 [==============================] - ETA: 0s - loss: 1.1899 - acc: 0.6680
Epoch 6: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
12/12 [==============================] - 18s 2s/step - loss: 1.1899 - acc: 0.6680 - val_loss: 0.9803

RESNET 152


In [ ]:
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K


In [ ]:
pre_trained_model = ResNet152(input_shape=(192, 256, 3), include_top=False, weights="imagenet")

234698864/234698864 [==============================] - 7s 0us/step


In [ ]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False

print(len(pre_trained_model.layers))

input_1
conv1_pad
conv1_conv
conv1_bn
conv1_relu
pool1_pad
pool1_pool
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu
conv2_block1_0_conv
conv2_block1_3_conv
conv2_block1_0_bn
conv2_block1_3_bn
conv2_block1_add
conv2_block1_out
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_conv
conv2_block2_2_bn
conv2_block2_2_relu
conv2_block2_3_conv
conv2_block2_3_bn
conv2_block2_add
conv2_block2_out
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_conv
conv2_block3_2_bn
conv2_block3_2_relu
conv2_block3_3_conv
conv2_block3_3_bn
conv2_block3_add
conv2_block3_out
conv3_block1_1_conv
conv3_block1_1_bn
conv3_block1_1_relu
conv3_block1_2_conv
conv3_block1_2_bn
conv3_block1_2_relu
conv3_block1_0_conv
conv3_block1_3_conv
conv3_block1_0_bn
conv3_block1_3_bn
conv3_block1_add
conv3_block1_out
conv3_block2_1_conv
conv3_block2_1_bn
conv3_block2_1_relu
conv3_block2_2_conv
conv3_block2_2_bn
conv3_

In [ ]:

last_layer = pre_trained_model.get_layer('conv5_block3_out')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output


last layer output shape: (None, 6, 8, 2048)


In [ ]:
from tensorflow.keras.optimizers.legacy import Adam

In [ ]:
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = True

In [ ]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])


In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5,
                                            min_lr=0.000001, cooldown=2)


In [ ]:

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 192, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 198, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 96, 128, 64)          9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 96, 128, 64)          256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
batch_size = 64
epochs = 35
history = model.fit_generator(train_datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(x_test, y_test),
                              verbose = 1, steps_per_epoch=(x_train.shape[0] // batch_size),
                              validation_steps=(x_test.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

<ipython-input-29-145368143d90>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(x_train,y_train, batch_size=batch_size),


Epoch 1/35


WARNING:tensorflow:From <ipython-input-20-b0e0df1b87c2>:7: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.

Epoch 1/35
112/112 [==============================] - 177s 2s/step - loss: 2.0453 - acc: 0.6560 - val_loss: 4.6930 - val_acc: 0.7161 - lr: 1.0000e-04

Epoch 2/35
112/112 [==============================] - 172s 2s/step - loss: 0.7052 - acc: 0.7462 - val_loss: 0.6951 - val_acc: 0.8438 - lr: 1.0000e-04

Epoch 3/35
112/112 [==============================] - 172s 2s/step - loss: 0.6004 - acc: 0.7824 - val_loss: 0.5447 - val_acc: 0.8177 - lr: 1.0000e-04

Epoch 4/35
112/112 [==============================] - 172s 2s/step - loss: 0.5378 - acc: 0.8058 - val_loss: 0.6044 - val_acc: 0.7656 - lr: 1.0000e-04

Epoch 5/35
112/112 [==============================] - ETA: 0s - loss: 0.4741 - acc: 0.8297

Epoch 00005: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
112/112 [==============================] - 172s 2s/step - loss: 0.4741 - acc: 0.8297 - val_loss: 0.5369 - val_acc: 0.8073 - lr: 1.0000e-04

Epoch 6/35
112/112 [==============================] - 172s 2s/step - loss: 0.4239 - acc: 0.8414 - val_loss: 0.4825 - val_acc: 0.8177 - lr: 5.0000e-05

Epoch 7/35
112/112 [==============================] - 172s 2s/step - loss: 0.3953 - acc: 0.8533 - val_loss: 0.5618 - val_acc: 0.7969 - lr: 5.0000e-05

Epoch 8/35
112/112 [==============================] - 172s 2s/step - loss: 0.3631 - acc: 0.8655 - val_loss: 0.4683 - val_acc: 0.8255 - lr: 5.0000e-05

Epoch 9/35
112/112 [==============================] - ETA: 0s - loss: 0.3558 - acc: 0.8679
Epoch 00009: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
112/112 [==============================] - 173s 2s/step - loss: 0.3558 - acc: 0.8679 - val_loss: 0.5417 - val_acc: 0.7917 - lr: 5.0000e-05

Epoch 10/35
112/112 [==============================] - 173s 2s/step - loss: 0.3099 - acc: 0.8825 - val_loss: 0.5110 - val_acc: 0.8099 - lr: 2.5000e-05

Epoch 11/35
112/112 [==============================] - 172s 2s/step - loss: 0.2831 - acc: 0.8945 - val_loss: 0.5736 - val_acc: 0.7812 - lr: 2.5000e-05

Epoch 12/35
112/112 [==============================] - 172s 2s/step - loss: 0.2669 - acc: 0.9023 - val_loss: 0.6233 - val_acc: 0.7943 - lr: 2.5000e-05

Epoch 13/35
112/112 [==============================] - 172s 2s/step - loss: 0.2596 - acc: 0.9018 - val_loss: 0.5265 - val_acc: 0.8464 - lr: 2.5000e-05

Epoch 14/35
112/112 [==============================] - 172s 2s/step - loss: 0.2438 - acc: 0.9131 - val_loss: 0.6261 - val_acc: 0.7734 - lr: 2.5000e-05

Epoch 15/35
112/112 [==============================] - 172s 2s/step - loss: 0.2456 - acc: 0.9054 - val_loss: 0.7202 - val_acc: 0.7396 - lr: 2.5000e-05

Epoch 16/35
112/112 [==============================] - ETA: 0s - loss: 0.2326 - acc: 0.9131
Epoch 00016: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
112/112 [==============================] - 172s 2s/step - loss: 0.2326 - acc: 0.9131 - val_loss: 0.5255 - val_acc: 0.8203 - lr: 2.5000e-05

Epoch 17/35
112/112 [==============================] - 172s 2s/step - loss: 0.2003 - acc: 0.9240 - val_loss: 0.7193 - val_acc: 0.8125 - lr: 1.2500e-05

Epoch 18/35
112/112 [==============================] - 172s 2s/step - loss: 0.1869 - acc: 0.9299 - val_loss: 0.6105 - val_acc: 0.8229 - lr: 1.2500e-05

Epoch 19/35
112/112 [==============================] - 172s 2s/step - loss: 0.1844 - acc: 0.9351 - val_loss: 0.5729 - val_acc: 0.8333 - lr: 1.2500e-05

Epoch 20/35
112/112 [==============================] - ETA: 0s - loss: 0.1707 - acc: 0.9372
Epoch 00020: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
112/112 [==============================] - 172s 2s/step - loss: 0.1707 - acc: 0.9372 - val_loss: 0.8060 - val_acc: 0.7917 - lr: 1.2500e-05

Epoch 21/35
112/112 [==============================] - 173s 2s/step - loss: 0.1441 - acc: 0.9452 - val_loss: 0.7694 - val_acc: 0.7995 - lr: 6.2500e-06
Epoch 22/35
112/112 [==============================] - 173s 2s/step - loss: 0.1371 - acc: 0.9467 - val_loss: 0.6024 - val_acc: 0.8333 - lr: 6.2500e-06

Epoch 23/35
112/112 [==============================] - 173s 2s/step - loss: 0.1431 - acc: 0.9456 - val_loss: 0.6075 - val_acc: 0.8281 - lr: 6.2500e-06

Epoch 24/35
112/112 [==============================] - ETA: 0s - loss: 0.1363 - acc: 0.9495
Epoch 00024: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-06.
112/112 [==============================] - 173s 2s/step - loss: 0.1363 - acc: 0.9495 - val_loss: 0.7874 - val_acc: 0.7943 - lr: 6.2500e-06

Epoch 25/35
112/112 [==============================] - 173s 2s/step - loss: 0.1364 - acc: 0.9505 - val_loss: 0.6811 - val_acc: 0.8307 - lr: 3.1250e-06

Epoch 26/35
112/112 [==============================] - 172s 2s/step - loss: 0.1219 - acc: 0.9559 - val_loss: 0.6908 - val_acc: 0.8255 - lr: 3.1250e-06

Epoch 27/35
112/112 [==============================] - 173s 2s/step - loss: 0.1207 - acc: 0.9538 - val_loss: 0.8003 - val_acc: 0.8021 - lr: 3.1250e-06

Epoch 28/35
112/112 [==============================] - ETA: 0s - loss: 0.1221 - acc: 0.9552
Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-06.
112/112 [==============================] - 173s 2s/step - loss: 0.1221 - acc: 0.9552 - val_loss: 0.7449 - val_acc: 0.7891 - lr: 3.1250e-06

Epoch 29/35
112/112 [==============================] - 172s 2s/step - loss: 0.1117 - acc: 0.9590 - val_loss: 0.9148 - val_acc: 0.7865 - lr: 1.5625e-06

Epoch 30/35
112/112 [==============================] - 173s 2s/step - loss: 0.1086 - acc: 0.9598 - val_loss: 0.8066 - val_acc: 0.7891 - lr: 1.5625e-06

Epoch 31/35
112/112 [==============================] - 172s 2s/step - loss: 0.1133 - acc: 0.9597 - val_loss: 0.6822 - val_acc: 0.8307 - lr: 1.5625e-06

Epoch 32/35
112/112 [==============================] - ETA: 0s - loss: 0.1063 - acc: 0.9625
Epoch 00032: ReduceLROnPlateau reducing learning rate to 1e-06.
112/112 [==============================] - 173s 2s/step - loss: 0.1063 - acc: 0.9625 - val_loss: 0.6139 - val_acc: 0.8255 - lr: 1.5625e-06

Epoch 33/35
112/112 [==============================] - 173s 2s/step - loss: 0.1081 - acc: 0.9597 - val_loss: 0.8092 - val_acc: 0.8073 - lr: 1.0000e-06

Epoch 34/35
112/112 [==============================] - 173s 2s/step - loss: 0.1035 - acc: 0.9638 - val_loss: 0.8199 - val_acc: 0.7969 - lr: 1.0000e-06

Epoch 35/35
112/112 [==============================] - 173s 2s/step - loss: 0.1080 - acc: 0.9583 - val_loss: 0.8706 - val_acc: 0.8021 - lr: 1.0000e-06